In [1]:
import requests
import json
import os
from auth_data import token # сервисный токен
from auth_data import tokenbgd # личный токен

import pandas as pd
import numpy as np

# <center> Получаем данные постов группы

In [2]:
# Запрашиваемые параметры
group_name = 'atheism' # Название группы взятое из ссылки
count_last_posts = 40 # Число последних постов

# Производим запрос
url_group = f"https://api.vk.com/method/wall.get?domain={group_name}&count={count_last_posts}&access_token={token}&v=5.81"
req = requests.get(url_group)
group_json = req.json()

# Создаем список с id номерами постов группы
post_id_list = list()
for i in range(len(group_json['response']['items'])):
    post_id_list.append(group_json['response']['items'][i]['id'])

# Проверяем существует ли директория с именем группы
if os.path.exists(f"{group_name}"):
    print(f"Директория с именем {group_name} уже существует!")
else:
    os.mkdir(group_name)
    # сохраняем данные в json файл, чтобы видеть структуру
    
with open(f"{group_name}/{group_name}.json", "w", encoding="utf-8") as file:
    json.dump(group_json, file, indent=4, ensure_ascii=False)

print("Номера постов:",post_id_list)

Директория с именем atheism уже существует!
Номера постов: [2794126, 2793803, 2793651, 2793616, 2793544, 2793477, 2793427, 2793321, 2793191, 2793003, 2792911, 2792632, 2792491, 2792311, 2792101, 2792039, 2791899, 2791772, 2791621, 2791384, 2791265, 2791185, 2791046, 2790974, 2790902, 2790861, 2790760, 2790678, 2790636, 2790522, 2790355, 2790229, 2790129, 2790016, 2789793, 2789663, 2789515, 2789339, 2789278, 2789201]


In [3]:
# Запрашиваемые параметры
owner_id = -63683472 #'atheism' # Название группы взятое из ссылки
post_id = 2794126 #post_id_list # id поста

# Производим запрос
url_posts = f"https://api.vk.com/method/wall.getComments?owner_id={owner_id}&post_id={post_id}&count=100&access_token={token}&v=5.81"
req = requests.get(url_posts)
post_id_json = req.json()

# Создаем список с пользователями и текстом комментариев
text = list()

for i in range(len(post_id_json['response']['items'])):
    text.append([
        post_id_json['response']['items'][i]['from_id'],
        post_id_json['response']['items'][i]['text']]
    )

# Проверяем существует ли директория с именем группы
if os.path.exists(f"{group_name}/{owner_id}/{post_id}")==False:
    os.makedirs(f"{group_name}/{owner_id}/{post_id}")

# Сохраняем данные в json файл, чтобы видеть структуру       
with open(f"{group_name}/{owner_id}/{post_id}/{post_id}.json", "w", encoding="utf-8") as file:
    json.dump(post_id_json, file, indent=4, ensure_ascii=False)

# Вносим полученные данные в таблицу
#users_data['id пользователя'],users_data['комментарий'] = text

# <center> Получение комментариев

In [4]:
# Запрашиваемые параметры
owner_id = -63683472 #'atheism' # Название группы взятое из ссылки
post_id = 2794126 #post_id_list # id поста

# Производим запрос
url_posts = f"https://api.vk.com/method/wall.getComments?owner_id={owner_id}&post_id={post_id}&count=100&access_token={token}&v=5.81"
req = requests.get(url_posts)
post_id_json = req.json()

# Создаем список с пользователями и текстом комментариев
text = list()

for i in range(len(post_id_json['response']['items'])):
    text.append([
        owner_id,                                           # id группы
        post_id,                                            # id поста
        post_id_json['response']['items'][i]['from_id'],    # id пользователя 
        post_id_json['response']['items'][i]['text']]       # текст комментария
    )

# Проверяем существует ли директория с именем группы
if os.path.exists(f"{group_name}/{owner_id}/{post_id}")==False:
    os.makedirs(f"{group_name}/{owner_id}/{post_id}")

# Сохраняем данные в json файл, чтобы видеть структуру       
with open(f"{group_name}/{owner_id}/{post_id}/{post_id}_text.json", "w", encoding="utf-8") as file:
    json.dump(text, file, indent=4, ensure_ascii=False)

# <center> Создаем таблицу, куда будем вносить данные о пользователях

In [5]:
users_data = pd.DataFrame(
    data = text,
    columns= ['id группы', 'id поста', 'id пользователя', 'Комментарий']
)

In [6]:
users_data.head()

,id группы,id поста,id пользователя,Комментарий
0,-63683472,2794126,-209889129,
1,-63683472,2794126,44857480,"Если этих людей завезти к нам, то уверен, смогут."
2,-63683472,2794126,329098773,Потому что там армяне и другие мигранты не кла...
3,-63683472,2794126,366024543,В Америке вместо асфальта используют бетон
4,-63683472,2794126,453007924,"Думала про людей, а там про асфальт...Афтор, в..."


# <center> Добавление данных о городе пользователя

In [7]:
# Напишем функцию, возвращающую имя пользователя
def user_first_name(user_id, token=token): 
    # Производим запрос
    url_posts = f"https://api.vk.com/method/users.get?user_ids={user_id}&fields=city&access_token={token}&v=5.81"
    req = requests.get(url_posts)
    users_info_json = req.json()    

    try:
        first_name = users_info_json['response'][0]['first_name']
        return first_name
    except IndexError:
        return 0     

# Напишем функцию, возвращающую фамилию пользователя
def user_last_name(user_id, token=token): 
    # Производим запрос
    url_posts = f"https://api.vk.com/method/users.get?user_ids={user_id}&fields=city&access_token={token}&v=5.81"
    req = requests.get(url_posts)
    users_info_json = req.json()    

    try:
        last_name = users_info_json['response'][0]['last_name']
        return last_name
    except IndexError:
        return 0    

# Напишем функцию, возвращающую город пользователя
def user_city(user_id, token=token): 
    # Производим запрос
    url_posts = f"https://api.vk.com/method/users.get?user_ids={user_id}&fields=city&access_token={token}&v=5.81"
    req = requests.get(url_posts)
    users_info_json = req.json()    

    try:        
        city = users_info_json['response'][0]['city']['title'] 
        return city
    except KeyError:
        return ''
    except IndexError:
        return 0    
    

In [8]:
users_data['Имя']       = users_data['id пользователя'].apply(user_first_name)
users_data['Фамилия']   = users_data['id пользователя'].apply(user_last_name)
users_data['Город']     = users_data['id пользователя'].apply(user_city)

In [15]:
# Напишем функцию, возвращающую имя пользователя
def user_info(user_id, token=token): 
    # Производим запрос
    url_posts = f"https://api.vk.com/method/users.get?user_ids={user_id}&fields=city&access_token={token}&v=5.81"
    req = requests.get(url_posts)
    users_info_json = req.json()    

    try:
        first_name = users_info_json['response'][0]['first_name']
        last_name = users_info_json['response'][0]['last_name']
        city = users_info_json['response'][0]['city']['title']         
    except KeyError:
        city = ''
    except IndexError:
        return '', '', ''   

    return first_name, last_name, city

In [16]:
for i in range(users_data.shape[0]):
    users_data['Имя'].iloc[i], users_data['Фамилия'].iloc[i], users_data['Город'].iloc[i] = user_info(users_data['id пользователя'].iloc[i])

c:\Users\mish9\.conda\envs\sf\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [9]:
# Генерируем ссылку на пост с комментариями
def url_to_post(id_group, id_post):
    return f"https://vk.com/atheism?w=wall{id_group}_{id_post}"

for i in range(users_data.shape[0]):
    id_group = users_data['id группы'].iloc[i]
    id_post = users_data['id поста'].iloc[i]
    users_data['Ссылка на пост'].iloc[i] = url_to_post(id_group, id_post)

KeyError: 'Ссылка на пост'

In [ ]:
users_data

,id группы,id поста,id пользователя,Комментарий,Имя,Фамилия,Город
0,-63683472,2794126,-209889129,,0,0,0
1,-63683472,2794126,44857480,"Если этих людей завезти к нам, то уверен, смогут.",Alexander,Andreev,Tolyatti
2,-63683472,2794126,329098773,Потому что там армяне и другие мигранты не кла...,Artem,Artemov,
3,-63683472,2794126,366024543,В Америке вместо асфальта используют бетон,Valery,Solovyev,Sukhinichi
4,-63683472,2794126,453007924,"Думала про людей, а там про асфальт...Афтор, в...",Dzhek,Ivanova,
...,...,...,...,...,...,...,...
86,-63683472,2794126,233378385,"[id381038185|Диана], Ну и хорошо что не пересе...",Vladimir,Suvorov,Chelyabinsk
87,-63683472,2794126,734006104,Ну реально России далеко до дорог в США\nдаже ...,Bartolomeu,Dias-De-Novaes,
88,-63683472,2794126,151506750,"[id381038185|Диана], ну да,весь сброд со всего...",Alex,Makey,Novosibirsk
89,-63683472,2794126,381038185,"[id151506750|Алекс], чем вам в Москве люди не ...",Diana,Potapova,


,id группы,id поста,id пользователя,Комментарий,Имя,Фамилия,Город,Ссылка на пост
0,-63683472,2794126,-209889129,,0,0,0,https://vk.com/atheism?w=wall-63683472_2794126
1,-63683472,2794126,44857480,"Если этих людей завезти к нам, то уверен, смогут.",Alexander,Andreev,Tolyatti,https://vk.com/atheism?w=wall-63683472_2794126
2,-63683472,2794126,329098773,Потому что там армяне и другие мигранты не кла...,Artem,Artemov,,https://vk.com/atheism?w=wall-63683472_2794126
3,-63683472,2794126,366024543,В Америке вместо асфальта используют бетон,Valery,Solovyev,Sukhinichi,https://vk.com/atheism?w=wall-63683472_2794126
4,-63683472,2794126,453007924,"Думала про людей, а там про асфальт...Афтор, в...",Dzhek,Ivanova,,https://vk.com/atheism?w=wall-63683472_2794126


In [ ]:
users_data.to_csv('users_data.csv', sep=';', decimal='.', index=True)